In [1]:
import pandas as pd
import os
from main import get_miner
from log_parsing import from_id_to_template
import ast
from datetime import datetime

In [2]:
df = pd.read_csv(f"output/features_labeled_4llm_step2.csv")

In [3]:
df.head()

,abnormal_time,inject_pod,inject_time,inject_type,normal_time,result_list_inhealthy_lessoften,result_list_inhealthy_moreoften,root_cause,inject_comp_tdiff,nezha_rank_050
0,2022-08-22 03:54,frontend-579b9bff58-t2dbm,2022-08-22 03:53:54,cpu_contention,2022-08-22 03:51,"[{'score': -0.5400894187779434, 'deepth': 3, '...","[{'score': 1.0, 'deepth': 1, 'pod': 'frontend-...",['CpuUsageRate(%)'],1,1
1,2022-08-22 03:55,frontend-579b9bff58-t2dbm,2022-08-22 03:53:54,cpu_contention,2022-08-22 03:51,"[{'score': -1.0, 'deepth': 1, 'pod': 'frontend...","[{'score': 1.0, 'deepth': 1, 'pod': 'frontend-...",['CpuUsageRate(%)'],2,1
2,2022-08-22 04:03,frontend-579b9bff58-t2dbm,2022-08-22 04:02:07,return,2022-08-22 03:51,"[{'score': -1.0, 'deepth': 1, 'pod': 'frontend...","[{'score': 1.0, 'deepth': 4, 'pod': 'adservice...","['Serving product page started', 'GetProduct s...",1,2
3,2022-08-22 04:04,frontend-579b9bff58-t2dbm,2022-08-22 04:02:07,return,2022-08-22 03:51,"[{'score': -1.0, 'deepth': 1, 'pod': 'frontend...","[{'score': 1.0, 'deepth': 4, 'pod': 'adservice...","['Serving product page started', 'GetProduct s...",2,2
4,2022-08-22 04:11,frontend-579b9bff58-t2dbm,2022-08-22 04:10:20,cpu_consumed,2022-08-22 03:51,"[{'score': -1.0, 'deepth': 1, 'pod': 'frontend...","[{'score': 0.6055825242718447, 'deepth': 2, 'p...",['CpuUsageRate(%)'],1,4


In [5]:
log_template_miner = get_miner()

In [6]:
df['result_list_inhealthy_lessoften'] = df['result_list_inhealthy_lessoften'].apply(lambda x: ast.literal_eval(x))
df['result_list_inhealthy_moreoften'] = df['result_list_inhealthy_moreoften'].apply(lambda x: ast.literal_eval(x))

In [7]:
def format_result_list(result_list):
    """
    Format result list for LLM comprehension
    """
    for i in range(len(result_list)):
        entry = result_list[i]
        if 'events_actual' in entry:
            entry['events_actual'] = [from_id_to_template(int(entry['events_actual'].split('_')[0]), log_template_miner),
                                      from_id_to_template(int(entry['events_actual'].split('_')[1]), log_template_miner)]
        if 'events_expected' in entry:
            entry['events_expected'] = [from_id_to_template(int(entry['events_expected'].split('_')[0]), log_template_miner),
                                      from_id_to_template(int(entry['events_expected'].split('_')[1]), log_template_miner)]
    
    return result_list

In [7]:
df['result_list_inhealthy_lessoften'] = df['result_list_inhealthy_lessoften'].apply(format_result_list)

In [8]:
df['result_list_inhealthy_moreoften'] = df['result_list_inhealthy_moreoften'].apply(format_result_list)

In [9]:
pd.set_option("display.max_colwidth", None)
df[['result_list_inhealthy_lessoften']].head()

,result_list_inhealthy_lessoften
0,"[{'score': -0.5400894187779434, 'deepth': 3, 'pod': 'currencyservice-cf787dd48-vpjrd', 'events_expected': ['TraceID: <:TRACEID:> SpanID: <:SPANID:> Received conversion request', 'currencyservice hipstershop.CurrencyService/GetCurrencyData start'], 'events_actual': ['TraceID: <:TRACEID:> SpanID: <:SPANID:> Received conversion request', 'currencyservice hipstershop.CurrencyService/Carry start']}, {'score': -0.5384615384615384, 'deepth': 3, 'pod': 'currencyservice-cf787dd48-vpjrd', 'events_expected': ['TraceID: <:TRACEID:> SpanID: <:SPANID:> Getting supported currencies...', 'currencyservice grpc.hipstershop.CurrencyService/GetSupportedCurrencies end'], 'events_actual': ['TraceID: <:TRACEID:> SpanID: <:SPANID:> Getting supported currencies...', 'currencyservice hipstershop.CurrencyService/GetCurrencyData start']}, {'score': -0.5315315315315315, 'deepth': 4, 'pod': 'adservice-5f6585d649-fnmft', 'resource_alert': 'MemoryUsageRate(%)', 'events_expected': ['<:NUM:>:<:NUM:>:<:NUM:>.<:NUM:> INFO - TraceID: <:TRACEID:> SpanID: <:SPANID:> Received ad request (context words=<:ITEM:>', 'adservice hipstershop.AdService/<:SPANID:> start'], 'events_actual': ['<:NUM:>:<:NUM:>:<:NUM:>.<:NUM:> INFO - TraceID: <:TRACEID:> SpanID: <:SPANID:> Received ad request (context words=<:ITEM:>', '<:NUM:>:<:NUM:>:<:NUM:>.<:NUM:> INFO - TraceID: <:TRACEID:> SpanID: <:SPANID:> No context provided. Constructe random Ads start']}, {'score': -0.518854415274463, 'deepth': 2, 'pod': 'currencyservice-cf787dd48-vpjrd', 'events_expected': ['currencyservice grpc.hipstershop.CurrencyService/Convert start', 'currencyservice hipstershop.CurrencyService/Carry start'], 'events_actual': ['currencyservice grpc.hipstershop.CurrencyService/Convert start', 'currencyservice hipstershop.CurrencyService/GetCurrencyData start']}, {'score': -0.5053908355795148, 'deepth': 1, 'pod': 'frontend-579b9bff58-t2dbm', 'resource_alert': 'CpuUsageRate(%)', 'events_expected': ['TraceID: <:TRACEID:> SpanID: <:SPANID:> View user cart start', 'frontend hipstershop.RecommendationService/ListRecommendations start'], 'events_actual': ['TraceID: <:TRACEID:> SpanID: <:SPANID:> View user cart start', 'frontend hipstershop.ProductCatalogService/GetProduct start']}]"
1,"[{'score': -1.0, 'deepth': 1, 'pod': 'frontend-579b9bff58-t2dbm', 'resource_alert': 'CpuUsageRate(%)', 'events_actual': ['frontend hipstershop.Frontend/Recv. start', 'CpuUsageRate(%)']}, {'score': -0.5517241379310345, 'deepth': 3, 'pod': 'currencyservice-cf787dd48-vpjrd', 'events_expected': ['TraceID: <:TRACEID:> SpanID: <:SPANID:> Received conversion request', 'TraceID: <:TRACEID:> SpanID: <:SPANID:> Conversion request successful'], 'events_actual': ['TraceID: <:TRACEID:> SpanID: <:SPANID:> Received conversion request', 'currencyservice hipstershop.CurrencyService/GetCurrencyData start']}, {'score': -0.5172413793103449, 'deepth': 5, 'pod': 'checkoutservice-578fcf4766-9csqn', 'events_actual': ['TraceID: <:TRACEID:> SpanID: <:SPANID:> Start query cost of products<:ITEM:>', 'checkoutservice hipstershop.ProductCatalogService/GetProduct start']}, {'score': -0.5172413793103449, 'deepth': 5, 'pod': 'checkoutservice-578fcf4766-9csqn', 'events_actual': ['TraceID: <:TRACEID:> SpanID: <:SPANID:> Start query cost of products<:ITEM:>', 'checkoutservice hipstershop.CheckoutService/ConvertCurrency start']}]"
2,"[{'score': -1.0, 'deepth': 1, 'pod': 'frontend-579b9bff58-t2dbm', 'events_expected': ['TraceID: <:TRACEID:> SpanID: <:SPANID:> Serving product page started', 'frontend hipstershop.RecommendationService/ListRecommendations start'], 'events_actual': ['TraceID: <:TRACEID:> SpanID: <:SPANID:> Serving product page started', 'TraceID: <:TRACEID:> SpanID: <:SPANID:> Product id not specified']}, {'score': -0.5735294117647058, 'deepth': 1, 'pod': 'frontend-579b9bff58-t2dbm', 'events_expected': ['TraceID: <:TRACEID:> SpanID: <:SPANID:> Request started', 'TraceID: <:TRACEID:> SpanID: <:SPANID:> Adding to cart started'], '

In [10]:
df.columns

Index(['abnormal_time', 'inject_pod', 'inject_time', 'inject_type',
       'normal_time', 'result_list_inhealthy_lessoften',
       'result_list_inhealthy_moreoften', 'root_cause', 'inject_comp_tdiff',
       'nezha_rank_050'],
      dtype='object')

In [11]:
df.inject_pod.unique()

array(['frontend-579b9bff58-t2dbm', 'cartservice-579f59597d-wc2lz',
       'checkoutservice-578fcf4766-9csqn',
       'currencyservice-cf787dd48-vpjrd', 'emailservice-55fdc5b988-f6xth',
       'paymentservice-9cdb6588f-554sm',
       'productcatalogservice-668d5f85fb-wckp8',
       'recommendationservice-6cfdd55578-gfj6q',
       'shippingservice-7b598fc7d-lmggd', 'adservice-5f6585d649-fnmft',
       'productcatalogservice-668d5f85fb-jhwx9'], dtype=object)

In [13]:
df.to_csv(f"output/features_labeled_4llm_step3.csv", index=False)